In [ ]:
%pip install langchain_experimental

## State

In [2]:
from langgraph.graph import MessagesState
from typing_extensions import Any

class State(MessagesState):
    question: str
    dataset: Any
    code: str
    is_executable: bool
    result: str
    answer: str

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o', temperature=0)

In [4]:
from langchain_core.tools import Tool
from langchain.tools import tool
from langchain_experimental.utilities import PythonREPL
from langgraph.prebuilt import ToolNode

## Nodes

In [ ]:
from typing_extensions import TypedDict, Annotated

class CodeBlock(TypedDict):
    code: Annotated[str, ..., '실행 가능한 파이썬 코드']

def generate_code(state: State):
    prompt = f'''
사용자 질문과 데이터셋을 제공할거야. 사용자 질문에 답변하기 위한 파이썬 코드를 생성해 줘.
코드는 간단할수록 좋고, numpy, pandas, scikit-learn, scipy 가 설치되어있으니 편하게 써.
대신 사용자의 파일/폴더를 건들일 수 있는 코드는 절대 작성하면 안돼.
---
질문: {state['question']}
---
데이터셋: {state['dataset']}
---
코드:
    '''
    structured_llm = llm.with_structured_output(CodeBlock)
    result = structured_llm.invoke(prompt)

    return {'code': result['code']}

code = generate_code({
    'question': '평균을 구해줘',
    'dataset': [1, 2, 3, 4, 5]
})['code']


def execute_code(state: State):
    python_repl = PythonREPL()
    repl_tool = Tool(
        name='python_repl',
        description='A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.',
        func=python_repl.run,
    )
    result = repl_tool.invoke(state['code'])
    return {'result': result}


Python REPL can execute arbitrary code. Use with caution.


{'result': 'The mean of the dataset is: 3.0\n'}

## Routers

## Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

builder = StateGraph(MessagesState)
# builder.add_node()

memory = InMemorySaver()
graph = builder.compile(checkpointer=memory)

In [ ]:
config = {
    'configurable': {'thread_id': '1'}
}
question = '현재 데이터의 평균과 표준편차를 구해줘'
graph.invoke({
    'question': question,
    'dataset': [1, 2, 3, 4, 5, 6, 7, 8]
}, config)